## Setup

Run this cell first to provision a cloud machine and set up Drake. 

This may take a minute or two the first time, but once a machine is set up it should be good for about 12 hours. 

In [1]:
import importlib
import sys
import os
from urllib.request import urlretrieve
import subprocess
import shutil

drake_url = "https://drake-packages.csail.mit.edu/tmp/drake-0.27.0-pip-bionic.tar.gz"
assert 'google.colab' in sys.modules, "This notebook is meant to be run in google colab!"

if importlib.util.find_spec('pydrake') is None:
    # We're in colab and don't have pydrake, so install it on the cloud machine.
    if os.path.isdir('/opt/drake'):
        shutil.rmtree('/opt/drake')
    print("Installing Drake")
    urlretrieve(drake_url, 'drake.tar.gz')
    subprocess.run(['mkdir', '/opt/drake'])
    subprocess.run(['tar', '-xzf', 'drake.tar.gz', '-C', '/opt/drake'], check=True)
    
    print("Installing other dependencies")
    subprocess.run(["pip3", "install", "meshcat"])
    subprocess.run(['apt-get', 'update', '-o', 'APT::Acquire::Retries=4', '-qq'], check=True)
    with open("/opt/drake/share/drake/setup/packages-bionic.txt", "r") as f:
        packages = f.read().splitlines()
    subprocess.run(['apt-get', 'install', '-o',
                    'APT::Acquire::Retries=4', '-o', 'Dpkg::Use-Pty=0',
                    '-qy', '--no-install-recommends'] + packages,
                    check=True)
    
    v = sys.version_info
    path = f"/opt/drake/lib/python{v.major}.{v.minor}/site-packages"
    if importlib.util.find_spec('pydrake') is None:
        sys.path.append(path)

# Clone our github repo
target_path = '/opt/passivity_cbf_demo'
if not os.path.isdir(target_path):
    print("Cloning github repo")
    subprocess.run(['git','clone','https://github.com/vincekurtz/passivity_cbf_demo.git',target_path])
sys.path.append(target_path)

print("Done!")


Installing Drake
Installing other dependencies
Cloning github repo
Done!


In [8]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/opt/drake/lib/python3.7/site-packages',
 '/opt/passivity_cbf_demo',
 '/opt/passivity_cbf_demo']

## About This Notebook

While you're waiting, here's what this is all about...

Now we can start by setting some initial parameters

In [15]:
import numpy as np
import matplotlib.pyplot as plt 
import os
from pydrake.all import *
from reduced_order_model import ReducedOrderModelPlant
from controller import Gen3Controller
from planners import GuiPlanner

sim_time = np.inf
dt = 3e-3
target_realtime_rate = 1.0 

# Initial joint angles
q0 = np.array([0.0,0,np.pi/2,-np.pi/2,0.0,-np.pi/2,0])

# initial end-effector pose
x0 = np.array([np.pi-0.5,  
               0,
               np.pi/2,
               0.2,
               0.3,
               0.5])


In [20]:
# Find the (local) description file relative to drake
robot_description_path = target_path + "/models/gen3_7dof/urdf/GEN3_URDF_V12.urdf"
drake_path = getDrakePath()
robot_description_file = "drake/" + os.path.relpath(robot_description_path, start=drake_path)

# Set up the diagram and MultibodyPlant
builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())
plant = builder.AddSystem(MultibodyPlant(time_step=dt))
plant.RegisterAsSourceForSceneGraph(scene_graph)

# Create a "controllable" plant, which has access only to the robot arm and gripper,
# and not any data about other objects in the scene
c_plant = MultibodyPlant(time_step=dt)

# Load the robot arm model from a urdf file
robot_urdf = FindResourceOrThrow(robot_description_file)
gen3 = Parser(plant=plant).AddModelFromFile(robot_urdf,"gen3")
c_gen3 = Parser(plant=c_plant).AddModelFromFile(robot_urdf,"gen3")

# Load the gripper model from a urdf file
gripper_file = "drake/" + os.path.relpath(target_path + "/models/hande_gripper/urdf/robotiq_hande.urdf", start=drake_path)
gripper_urdf = FindResourceOrThrow(gripper_file)
gripper = Parser(plant=plant).AddModelFromFile(gripper_urdf,"gripper")
c_gripper = Parser(plant=c_plant).AddModelFromFile(gripper_urdf,"gripper")

print(c_gripper)

ModelInstanceIndex(3)
